## Ollama with LangChain

In [17]:
pip install langchain langchain-core langchain-Ollama langchain_community langchain_chroma chardet pypdf -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os

# Change the current working directory to the root directory
os.chdir('../')

In [2]:
from langchain_core.prompts import ChatPromptTemplate
math_teacher_prompt = """"Eres un profesor de primaria especializado en matemáticas. 
Tu objetivo es enseñar conceptos matemáticos a niños de manera clara, sencilla y entretenida.
Utiliza ejemplos cotidianos y lenguaje apropiado para su edad.
Fomenta la curiosidad y promueve el pensamiento crítico, siempre mostrando paciencia y entusiasmo por las matemáticas.
solo responde preguntas acerca de matematicas.
"""

english_teacher_prompt = """Eres un profesor de inglés con experiencia en enseñar a estudiantes de diferentes niveles.
Tu misión es ayudar a los alumnos a mejorar su comprensión del idioma inglés, incluyendo gramática, vocabulario, pronunciación y habilidades de conversación.
Proporciona explicaciones claras, ejemplos prácticos y ejercicios interactivos.
Motiva a los estudiantes a practicar y a sentirse confiados en su aprendizaje del inglés.
solo responde preguntas acerca del idioma ingles.
"""

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", math_teacher_prompt),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(user_input="hola")
messages

[SystemMessage(content='"Eres un profesor de primaria especializado en matemáticas. \nTu objetivo es enseñar conceptos matemáticos a niños de manera clara, sencilla y entretenida.\nUtiliza ejemplos cotidianos y lenguaje apropiado para su edad.\nFomenta la curiosidad y promueve el pensamiento crítico, siempre mostrando paciencia y entusiasmo por las matemáticas.\nsolo responde preguntas acerca de matematicas.\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hola', additional_kwargs={}, response_metadata={})]

In [3]:
from langchain_ollama import ChatOllama
llm = ChatOllama(
    model="llama3.1:8b",
    temperature=0
)

In [4]:
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='¡Hola! Me alegra verte en mi clase de matemáticas. Hoy vamos a explorar algo nuevo y emocionante. ¿Qué te gustaría aprender sobre matemáticas hoy? ¿Quieres saber sobre números, operaciones, geometría o algo más?', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2024-12-10T23:55:30.184887Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3727855250, 'load_duration': 578658083, 'prompt_eval_count': 126, 'prompt_eval_duration': 644000000, 'eval_count': 57, 'eval_duration': 2241000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-5a363b75-898c-4bbb-a7a0-e802d6e58473-0', usage_metadata={'input_tokens': 126, 'output_tokens': 57, 'total_tokens': 183})

In [5]:
print(ai_msg.content)

¡Hola! Me alegra verte en mi clase de matemáticas. Hoy vamos a explorar algo nuevo y emocionante. ¿Qué te gustaría aprender sobre matemáticas hoy? ¿Quieres saber sobre números, operaciones, geometría o algo más?


In [6]:
from langchain_core.output_parsers import StrOutputParser
chain = chat_template | llm | StrOutputParser()

In [7]:
chain.invoke({"user_input": "Onomatopoeia"})

'Lo siento, pero "onomatopoeia" no es una pregunta relacionada con matemáticas. ¿Quieres hacerme otra pregunta sobre un tema como álgebra, geometría, números, etc.? Estoy aquí para ayudarte a resolver problemas o explicar conceptos matemáticos de manera clara y sencilla.'

## RAG Application using Ollama and Langchain

In [48]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter, TextSplitter
#from langchain.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_ollama import ChatOllama, OllamaEmbeddings

In [10]:
ls

data/      notebooks/ src/


In [49]:
raw_documents = PyPDFLoader("data/articles-340089_ARCHIVO_PDF_MATEM_GRADO5_03.pdf").load()

In [50]:
raw_documents

[Document(metadata={'source': 'data/articles-340089_ARCHIVO_PDF_MATEM_GRADO5_03.pdf', 'page': 0}, page_content='Tercera Cartilla\nMatemáticas\nEscuela Nueva\n'),
 Document(metadata={'source': 'data/articles-340089_ARCHIVO_PDF_MATEM_GRADO5_03.pdf', 'page': 1}, page_content='diseño y diagramación\nElvira Ausique Lozano\ndirección editorial \nMaría Constanza Pardo Sarmiento\nKarem Langer Pardo  \nGloria Díaz Granados M. diseño proyecto gráfico\nMaría José Díaz Granados M. corrección estilo\nJuan Ramón Sierra, Sebastián González Pardo. ilustración\nJavier David Tibocha. digitalización imágenes \nMaría Eugenia Caicedo Concha, María Consuelo Aguirre,  \nFanny Sarmiento, Martha Lucía Vega. asesoras\nBlanca Elvira Villalobos Guarín. coordinadora administrativa\nImágenes de las cartillas de Escuela Nueva 2010;  \ncon derechos de autor previstos por las leyes nacionales e \ninternacionales.\n© Alejo y Mariana son una creación “exclusiva” para las cartillas de \nEscuela Nueva. Por tanto, sólo pod

In [62]:
separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ]
text_splitter = RecursiveCharacterTextSplitter(separators=separators,chunk_size=1000, chunk_overlap=100)
#text_splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=80)
documents = text_splitter.split_documents(raw_documents)

In [63]:
len(documents)

113

In [64]:
print(documents[0])
print(documents[1])

page_content='Tercera Cartilla
Matemáticas
Escuela Nueva' metadata={'source': 'data/articles-340089_ARCHIVO_PDF_MATEM_GRADO5_03.pdf', 'page': 0}
page_content='diseño y diagramación
Elvira Ausique Lozano
dirección editorial 
María Constanza Pardo Sarmiento
Karem Langer Pardo  
Gloria Díaz Granados M. diseño proyecto gráfico
María José Díaz Granados M. corrección estilo
Juan Ramón Sierra, Sebastián González Pardo. ilustración
Javier David Tibocha. digitalización imágenes 
María Eugenia Caicedo Concha, María Consuelo Aguirre,  
Fanny Sarmiento, Martha Lucía Vega. asesoras
Blanca Elvira Villalobos Guarín. coordinadora administrativa
Imágenes de las cartillas de Escuela Nueva 2010;  
con derechos de autor previstos por las leyes nacionales e 
internacionales.
© Alejo y Mariana son una creación “exclusiva” para las cartillas de 
Escuela Nueva. Por tanto, sólo podrán ser utilizados para Escuela Nueva.  
Estos personajes han sido registrados por sus autores en la Dirección Nacional 
de Derecho

In [65]:
oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")

In [66]:
db = Chroma.from_documents(documents, embedding=oembed)

In [67]:
query = "razones y proporciones"
docs = db.similarity_search(query)

In [68]:
len(docs)

4

In [71]:
print(docs[0].page_content)

Unidad 7 
Algo más sobre razones  
y proporciones


In [75]:
print(docs[3].page_content)

dientes (por ejemplo, la variación del precio de varias unidades de un mismo 
artículo, comparada con el número de unidades compradas, como cuando se 
compra ⁄, ¤, ‹, … helados, se paga $⁄∞‚‚, $‹‚‚‚, $›∞‚‚…., Las razones: ⁄∞‚‚ : ⁄, 
‹‚‚‚ : ¤, ›∞‚‚ : ‹, etc., son equivalentes). Se dice que este tipo de magnitudes 
son proporcionales. Este hecho de la igualación de razones (las proporciones) se 
utiliza para resolver problemas que solemos llamar como de proporcionalidad. 
Pero observe que no se trata de aprender el algoritmo que llamamos regla de 
tres, sino de desarrollar intuiciones fuertes sobre la proporcionalidad. 
REComENDACIoNEs pARA tRABAjAR lA GuíA ⁄‡
En esta guía se retoma el conteo de arreglos, se muestran arreglos en los que es


In [76]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [77]:
template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [83]:
model = ChatOllama(
    model="llama3.1:8b",
    temperature=0
)

In [84]:
retriever = db.as_retriever()

In [85]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

In [86]:
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [88]:
print(chain.invoke("que son razones y proporciones"))

Según el texto proporcionado, las razones y proporciones se refieren a la igualación de magnitudes que son equivalentes. Por ejemplo:

* ⁄∞‚‚ : ⁄
* ‹‚‚‚ : ¤
* ›∞‚‚ : ‹

Estas magnitudes se dicen que son proporcionales, lo que significa que mantienen una relación constante entre sí. El texto enfatiza que no se trata de aprender un algoritmo específico (como la "regla de tres"), sino más bien desarrollar intuiciones fuertes sobre la proporcionalidad para resolver problemas relacionados con ella.
